# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 


from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer
from config_observer.CKPD import ckpd_to_CkpdObsConfig

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents - DrFirst-JHU Collaboration Center/DrFirst-Project/2024-DrFirst-v4-SPACE/_DrFirst-AI-EgmPred-WorkSpace


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools


####################
TriggerCaseMethod = 'TrulicityRx'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'Rx',
 'case_id_columns': ['PID', 'ObsDT', 'PInvID', 'RxID'],
 'special_columns': ['PID', 'DT', 'PInvID', 'RxID', 'drug_name'],
 'convert_TriggerEvent_to_Caseset': <function TrulicityRx.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [3]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_args['case_id_columns'] = case_id_columns
cohort_args['ObsDTName'] = 'ObsDT'
cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

print(cohort_args)
print(TriggerCasePath)

df_case = pd.read_pickle(TriggerCasePath) # (1000)
df_case = df_case.sample(1000, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

{'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RawRootID': 'patient_id_encoded', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_Sequence': ['P', 'PInv', 'Rx', 'EgmAuthen', 'EgmCallPharm', 'EgmClick', 'EgmCopay', 'EgmEdu', 'EgmRmd'], 'RecName_to_PrtRecName': {'P': 'None', 'PInv': 'P', 'Rx': 'PInv', 'EgmAuthen': 'PInv', 'EgmCallPharm': 'PInv', 'EgmClick': 'PInv', 'EgmCopay': 'Rx', 'EgmEdu': 'Rx', 'EgmRmd': 'Rx'}, 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_r

Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'DT', 'drug_name'],
    num_rows: 1000
})


## [Step 2] RO: Record Observation & CasePhi


In [4]:
###########################
ROName_List = ['Rx-ObsPnt-InsCate']
name_CasePhi = 'AggMeanFeat'
###########################

In [5]:
from recfldtkn.obsname import convert_RONameList_to_COName

CaseObsName = convert_RONameList_to_COName(ROName_List, name_CasePhi)
print(CaseObsName)

ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat


In [6]:
# part of fetch_caseobs_Phi_tools
CO_Folder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CO_Folder): os.makedirs(CO_Folder)
print(CO_Folder)

../_Data/2-Data_CaseObs/ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [7]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1033176, 1009211, 1017193, 1027056, 1014849], 'ObsDT': [Timestamp('2023-09-25 13:31:48.690000'), Timestamp('2023-09-01 19:40:45.612000'), Timestamp('2023-09-08 12:58:32.934000'), Timestamp('2023-07-17 20:54:55.528000'), Timestamp('2023-09-12 01:17:17.377000')], 'PInvID': ['1033176-003', '1009211-001', '1017193-010', '1027056-002', '1014849-017'], 'RxID': ['1033176-003-000', '1009211-001-000', '1017193-010-000', '1027056-002-000', '1014849-017-002'], 'DT': [Timestamp('2023-09-25 13:31:48.690000'), Timestamp('2023-09-01 19:40:45.612000'), Timestamp('2023-09-08 12:58:32.934000'), Timestamp('2023-07-17 20:54:55.528000'), Timestamp('2023-09-12 01:17:17.377000')], 'drug_name': ['Trulicity', 'Trulicity', 'Trulicity', 'Trulicity', 'Trulicity']}


In [8]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1033176,
     'ObsDT': Timestamp('2023-09-25 13:31:48.690000'),
     'PInvID': '1033176-003',
     'RxID': '1033176-003-000',
     'DT': Timestamp('2023-09-25 13:31:48.690000'),
     'drug_name': 'Trulicity'},
 1: {'PID': 1009211,
     'ObsDT': Timestamp('2023-09-01 19:40:45.612000'),
     'PInvID': '1009211-001',
     'RxID': '1009211-001-000',
     'DT': Timestamp('2023-09-01 19:40:45.612000'),
     'drug_name': 'Trulicity'},
 2: {'PID': 1017193,
     'ObsDT': Timestamp('2023-09-08 12:58:32.934000'),
     'PInvID': '1017193-010',
     'RxID': '1017193-010-000',
     'DT': Timestamp('2023-09-08 12:58:32.934000'),
     'drug_name': 'Trulicity'},
 3: {'PID': 1027056,
     'ObsDT': Timestamp('2023-07-17 20:54:55.528000'),
     'PInvID': '1027056-002',
     'RxID': '1027056-002-000',
     'DT': Timestamp('2023-07-17 20:54:55.528000'),
     'drug_name': 'Trulicity'},
 4: {'PID': 1014849,
     'ObsDT': Timestamp('2023-09-12 01:17:17.377000'),
     'PInvID': '1014849-017',
     '

In [9]:
case_example = idx_to_examples[1]
case_example

{'PID': 1009211,
 'ObsDT': Timestamp('2023-09-01 19:40:45.612000'),
 'PInvID': '1009211-001',
 'RxID': '1009211-001-000',
 'DT': Timestamp('2023-09-01 19:40:45.612000'),
 'drug_name': 'Trulicity'}

## [Step 2] get_caseobs_id

In [10]:
import inspect
#################################################
def get_CO_id(case_example, CaseObsName, cohort_args):
    RootID = cohort_args['RootID']
    ObsDTName = cohort_args['ObsDTName']
    PIDValue = case_example[RootID]
    ObsDTValue = case_example[ObsDTName].isoformat()
    return f'{PIDValue}&{ObsDTValue}'

get_CO_id.fn_string = inspect.getsource(get_CO_id)
#################################################

In [11]:
caseobs_id = get_CO_id(case_example, CaseObsName, cohort_args)
caseobs_id # CO_id

'1009211&2023-09-01T19:40:45.612000'



## [Step 3] RO List and CasePhi

In [12]:
name_CasePhi 

'AggMeanFeat'

In [13]:
## we need to prepare these five arguments.
# CaseTkn     # <------- we have this.
# cohort_args # <------- we have this.

# RecObs_Name 
# rec_args
# column_names # attribute name of R^recordname

In [14]:
######### RecObs_Name
print(ROName_List) # RO_List
# Let's focus on one RO
ROName = ROName_List[0]
print(ROName)

['Rx-ObsPnt-InsCate']
Rx-ObsPnt-InsCate


In [15]:
# RO = 'Rx-Bf2M-InsCate'
# RO = 'Rx-Bf2M'
# RO = 'P-Zip3DemoNume'

## [Step4] RO Information

In [16]:
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

Rx ObsPnt InsCate


In [17]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
[i for i in rec_args]

Rx


['CohortInfo',
 'RawInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'RawRecID',
 'RecDT',
 'ParentRecName',
 'attr_cols',
 'pypath',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'recfldtkn_config_path',
 'yaml_file_path']

## [Step 5] Load ds_rec

In [18]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)
# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [19]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PInvID',
 'RxID',
 'prescription_id_encoded',
 'DT',
 'start_date',
 'invitation_date',
 'insurance_start_date',
 'written_date',
 'date_fdb_updated',
 'prescriber_npi',
 'ncpdp_id',
 'pharmacy_name_rx',
 'simple_pharmacy_name_rx',
 'specialty_pharmacy',
 'pharmacy_zip_code_3',
 'ndc_id',
 'drug_name',
 'drug_description',
 'generic_id',
 'top_200_branded_drugs',
 'top_50_generic_drugs',
 'brand_source',
 'fill_number',
 'refills_available',
 'quantity_unit',
 'quantity',
 'days_supply',
 'package_size',
 'package_units',
 'strength',
 'strength_units',
 'total_package_strength',
 'directions',
 'delivery_type',
 'legend_status',
 'show_rems_campaigns',
 'show_coupon_campaigns',
 'show_educational_campaigns',
 'show_internal_campaigns',
 'show_target_campaigns',
 'show_experimental_campaigns',
 'send_refill_reminder_messages',
 'send_renewal_reminder_messages',
 'supports_hippo_prices',
 'supports_hippo_prices65',
 'supports_copay_prices',
 'supports_hippo_prices_medicare',
 

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [20]:
ROName

'Rx-ObsPnt-InsCate'

In [21]:
from recfldtkn.obsname import parse_RecObsName
# RecName = RecObs_Name.split('-')[0]
base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
base_columns

['PID', 'RxID', 'DT']

In [22]:

RecName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['RecName']
FldName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['FldName']
RecFldTknName = RecName + '-' + FldName + 'Tkn'
# rec_args['FldTknInfo'][RecFldTknName]
RecFldTknName

'Rx-InsCateTkn'

In [23]:
val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']

In [24]:
final_cols = base_columns + val_cols
final_cols

['PID', 'RxID', 'DT', 'rx_bin', 'rx_pcn', 'plan_name']

In [25]:
###############################
def get_selected_columns(ROName, column_names, cohort_args, rec_args, CaseTkn):
    from recfldtkn.obsname import parse_RecObsName
    RecName = parse_RecObsName(ROName, cohort_args['ckpd_to_CkpdObsConfig'])['RecName']
    FldName = parse_RecObsName(ROName, cohort_args['ckpd_to_CkpdObsConfig'])['FldName']
    base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
    
    FldName_cols = [i for i in column_names if FldName in i]
    if len(FldName_cols) == 0: 
        parse_RecObsName(ROName)['FldName']
        RecFldTknName = RecName + '-' + FldName + 'Tkn'
        # rec_args['FldTknInfo'][RecFldTknName]
        
        val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']
        final_cols = base_columns + val_cols
    else:
        final_cols = base_columns + FldName_cols
    return final_cols

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [26]:
RecName

'Rx'

In [27]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?
get_selected_columns(ROName, column_names, cohort_args, rec_args, name_CasePhi)

['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx']

In [28]:
print(ROName_List)
print(name_CasePhi)
print(get_selected_columns)

['Rx-ObsPnt-InsCate']
AggMeanFeat
<function get_selected_columns at 0x18616ff60>


In [29]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
ROName_to_ROInfo = get_RecObsName_to_RecObsInfo(ROName_List, 
                                                name_CasePhi, 
                                                get_selected_columns,
                                                cohort_args, 
                                                cohort_args['ckpd_to_CkpdObsConfig'], 
                                                record_to_ds_rec, 
                                                record_to_ds_rec_info)
    
ROName = ROName_List[0] # Rx-Bf2M  
ROInfo = ROName_to_ROInfo[ROName] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)

[INFO:2024-04-09 12:19:15,435:(configfn.py@110 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline/config_recfldtkn/Record/Rx.yaml


In [30]:
[i for i in ROInfo]
# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'fldtkn_args',
 'fld_tokenizer_fn',
 'run_fldtkn_on_the_fly',
 'ds_rec',
 'df_rec_info']

In [31]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 571656
})

## [Step 7] get_CO_vocab

In [32]:
def get_CO_vocab(RecObsName_to_RecObsInfo):
    # phi_list
    RecFldName_list = list(set([RecObsInfo['RecName'] + '-' +RecObsInfo['FldName'] 
                                for RecObsName, RecObsInfo in RecObsName_to_RecObsInfo.items()
                                if RecObsInfo['FldName'] is not None ]
                                ))
    
    # print('RecFldName_list----->', RecFldName_list)
    # for current version: for any Phi, we only process one or zero phi. 
    assert len(RecFldName_list) <= 1

    ############################ tkn 
    RecObsName = [i for i in RecObsName_to_RecObsInfo][0]
    RecObsInfo = RecObsName_to_RecObsInfo[RecObsName]
    idx2tkn = RecObsInfo['FldIdx2Tkn'] + ['NoObs']   
    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab = {}
    CaseTknVocab['tid'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    CaseTknVocab['wgt'] = {}
    ############################
    
    return CaseTknVocab

get_CO_vocab.fn_string = inspect.getsource(get_CO_vocab)

In [33]:
CO_vocab = get_CO_vocab(ROName_to_ROInfo)
pprint(CO_vocab, sort_dicts=False)

{'tid': {'tid2tkn': {0: 'rx_bin_unk',
                     1: 'rx_bin_minor',
                     2: 'rx_bin_4336.0',
                     3: 'rx_bin_610097.0',
                     4: 'rx_bin_610014.0',
                     5: 'rx_bin_3858.0',
                     6: 'rx_bin_15581.0',
                     7: 'rx_bin_610502.0',
                     8: 'rx_bin_610011.0',
                     9: 'rx_bin_11552.0',
                     10: 'rx_bin_610494.0',
                     11: 'rx_bin_610239.0',
                     12: 'rx_bin_610279.0',
                     13: 'rx_bin_20115.0',
                     14: 'rx_bin_20099.0',
                     15: 'rx_bin_17010.0',
                     16: 'rx_bin_24251.0',
                     17: 'rx_bin_22659.0',
                     18: 'rx_bin_12833.0',
                     19: 'rx_bin_4915.0',
                     20: 'rx_bin_19595.0',
                     21: 'rx_bin_1553.0',
                     22: 'rx_bin_20107.0',
                     23:


## [Step 8] Fetch ds_rec

In [34]:
case_example = idx_to_examples[4]
case_example

{'PID': 1014849,
 'ObsDT': Timestamp('2023-09-12 01:17:17.377000'),
 'PInvID': '1014849-017',
 'RxID': '1014849-017-002',
 'DT': Timestamp('2023-09-12 01:17:17.377000'),
 'drug_name': 'Trulicity'}

In [35]:
ROName

'Rx-ObsPnt-InsCate'

In [36]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1033176,
     'ObsDT': Timestamp('2023-09-25 13:31:48.690000'),
     'PInvID': '1033176-003',
     'RxID': '1033176-003-000',
     'DT': Timestamp('2023-09-25 13:31:48.690000'),
     'drug_name': 'Trulicity'},
 1: {'PID': 1009211,
     'ObsDT': Timestamp('2023-09-01 19:40:45.612000'),
     'PInvID': '1009211-001',
     'RxID': '1009211-001-000',
     'DT': Timestamp('2023-09-01 19:40:45.612000'),
     'drug_name': 'Trulicity'},
 2: {'PID': 1017193,
     'ObsDT': Timestamp('2023-09-08 12:58:32.934000'),
     'PInvID': '1017193-010',
     'RxID': '1017193-010-000',
     'DT': Timestamp('2023-09-08 12:58:32.934000'),
     'drug_name': 'Trulicity'},
 3: {'PID': 1027056,
     'ObsDT': Timestamp('2023-07-17 20:54:55.528000'),
     'PInvID': '1027056-002',
     'RxID': '1027056-002-000',
     'DT': Timestamp('2023-07-17 20:54:55.528000'),
     'drug_name': 'Trulicity'},
 4: {'PID': 1014849,
     'ObsDT': Timestamp('2023-09-12 01:17:17.377000'),
     'PInvID': '1014849-017',
     '

In [37]:
for ROName, ROInfo in ROName_to_ROInfo.items():
    print(ROName, ':', [i for i in ROInfo])

Rx-ObsPnt-InsCate : ['rec_args', 'RecName', 'CkpdName', 'CkpdInfo', 'FldName', 'FldTknName', 'FldIdx2Tkn', 'fldtkn_args', 'fld_tokenizer_fn', 'run_fldtkn_on_the_fly', 'ds_rec', 'df_rec_info']


In [38]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import CaseObserverTransformer

get_Record_P = CaseObserverTransformer.get_Record_P
RecName_to_REC_P = get_Record_P(idx_to_examples, ROName_to_ROInfo)
for RecName, REC_P in RecName_to_REC_P.items():
    print(f'============ {RecName} ============')
    pprint(REC_P)

============ Rx ============
{1009211: Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 1
}),
 1014849: Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 28
}),
 1017193: Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 22
}),
 1027056: Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 6
}),
 1033176: Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 5
})}


## [Step 9] Fetch ROdf: $R_i^{recname}$

In [39]:
get_idx_to_RecObsName_to_RecObsDS = CaseObserverTransformer.get_idx_to_RecObsName_to_RecObsDS
idx_to_ROName_to_ROds = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, ROName_to_ROInfo, RecName_to_REC_P)

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    print('ROdf:', idx_to_ROName_to_ROds[idx]) # R_i^{RecNum, Ckpd}
    print('====================')


idx: 0
case: {'PID': 1033176, 'ObsDT': Timestamp('2023-09-25 13:31:48.690000'), 'PInvID': '1033176-003', 'RxID': '1033176-003-000', 'DT': Timestamp('2023-09-25 13:31:48.690000'), 'drug_name': 'Trulicity'}
ROdf: {'Rx-ObsPnt-InsCate': Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 1
})}

idx: 1
case: {'PID': 1009211, 'ObsDT': Timestamp('2023-09-01 19:40:45.612000'), 'PInvID': '1009211-001', 'RxID': '1009211-001-000', 'DT': Timestamp('2023-09-01 19:40:45.612000'), 'drug_name': 'Trulicity'}
ROdf: {'Rx-ObsPnt-InsCate': Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 1
})}

idx: 2
case: {'PID': 1017193, 'ObsDT': Timestamp('2023-09-08 12:58:32.934000'), 'PInvID': '1017193-010', 'RxID': '1017193-010-000', 'DT': Timestamp('2023-09-08 12:58:32.934000'), 'drug_name': 'Trulicity'}
ROdf: {'Rx-ObsPnt-InsCate': Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-Ins

In [40]:
[i for i in ROName_to_ROInfo]

['Rx-ObsPnt-InsCate']

In [41]:
CO_vocab

{'tid': {'tid2tkn': {0: 'rx_bin_unk',
   1: 'rx_bin_minor',
   2: 'rx_bin_4336.0',
   3: 'rx_bin_610097.0',
   4: 'rx_bin_610014.0',
   5: 'rx_bin_3858.0',
   6: 'rx_bin_15581.0',
   7: 'rx_bin_610502.0',
   8: 'rx_bin_610011.0',
   9: 'rx_bin_11552.0',
   10: 'rx_bin_610494.0',
   11: 'rx_bin_610239.0',
   12: 'rx_bin_610279.0',
   13: 'rx_bin_20115.0',
   14: 'rx_bin_20099.0',
   15: 'rx_bin_17010.0',
   16: 'rx_bin_24251.0',
   17: 'rx_bin_22659.0',
   18: 'rx_bin_12833.0',
   19: 'rx_bin_4915.0',
   20: 'rx_bin_19595.0',
   21: 'rx_bin_1553.0',
   22: 'rx_bin_20107.0',
   23: 'rx_bin_610591.0',
   24: 'rx_bin_610455.0',
   25: 'rx_bin_610602.0',
   26: 'rx_bin_23880.0',
   27: 'rx_bin_10579.0',
   28: 'rx_bin_18902.0',
   29: 'rx_bin_610593.0',
   30: 'rx_bin_12312.0',
   31: 'rx_bin_3585.0',
   32: 'rx_pcn_unk',
   33: 'rx_pcn_minor',
   34: 'rx_pcn_ADV',
   35: 'rx_pcn_9999',
   36: 'rx_pcn_MEDDADV',
   37: 'rx_pcn_A4',
   38: 'rx_pcn_03200000',
   39: 'rx_pcn_MCAIDADV',
   40: '

In [42]:
idx = 3
case_example = idx_to_examples[idx]
print('case_example:', case_example)
ROName_to_ROds = idx_to_ROName_to_ROds[idx] # RO_to_ROdf # RO_ij
print('RO_to_ROdf (RO_ij):', ROName_to_ROds)
print([i for i in ROName_to_ROds])

case_example: {'PID': 1027056, 'ObsDT': Timestamp('2023-07-17 20:54:55.528000'), 'PInvID': '1027056-002', 'RxID': '1027056-002-000', 'DT': Timestamp('2023-07-17 20:54:55.528000'), 'drug_name': 'Trulicity'}
RO_to_ROdf (RO_ij): {'Rx-ObsPnt-InsCate': Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 1
})}
['Rx-ObsPnt-InsCate']


# [Step 10]: Develop $\Phi$. 

In [43]:
case_example

{'PID': 1027056,
 'ObsDT': Timestamp('2023-07-17 20:54:55.528000'),
 'PInvID': '1027056-002',
 'RxID': '1027056-002-000',
 'DT': Timestamp('2023-07-17 20:54:55.528000'),
 'drug_name': 'Trulicity'}

In [44]:
ROName_to_ROds

{'Rx-ObsPnt-InsCate': Dataset({
     features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
     num_rows: 1
 })}

In [45]:
ROName_to_ROInfo

{'Rx-ObsPnt-InsCate': {'rec_args': {'CohortInfo': {'RawData2023Nov_Trulicity': {'RxBase': {'raw_data_path': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity/prescription_df_de_identified.csv',
      'RawRootID': 'patient_id_encoded',
      'RecNumColumn': 'prescription',
      'raw_columns': ['patient_id_encoded',
       'invitation_id_encoded',
       'prescription_id_encoded',
       'invitation_date',
       'prescriber_npi',
       'state_code',
       'ncpdp_id',
       'pharmacy_name_rx',
       'simple_pharmacy_name_rx',
       'specialty_pharmacy',
       'pharmacy_zip_code_3',
       'ndc_id',
       'drug_name',
       'drug_description',
       'generic_id',
       'top_200_branded_drugs',
       'top_50_generic_drugs',
       'brand_source',
       'fill_number',
       'refills_available',
       'quantity',
       'days_supply',
       'package_size',
       'package_units',
       'quantity_unit',
       'strength',
       'strength_units',
       'total_package_strength

In [46]:
CO_vocab

{'tid': {'tid2tkn': {0: 'rx_bin_unk',
   1: 'rx_bin_minor',
   2: 'rx_bin_4336.0',
   3: 'rx_bin_610097.0',
   4: 'rx_bin_610014.0',
   5: 'rx_bin_3858.0',
   6: 'rx_bin_15581.0',
   7: 'rx_bin_610502.0',
   8: 'rx_bin_610011.0',
   9: 'rx_bin_11552.0',
   10: 'rx_bin_610494.0',
   11: 'rx_bin_610239.0',
   12: 'rx_bin_610279.0',
   13: 'rx_bin_20115.0',
   14: 'rx_bin_20099.0',
   15: 'rx_bin_17010.0',
   16: 'rx_bin_24251.0',
   17: 'rx_bin_22659.0',
   18: 'rx_bin_12833.0',
   19: 'rx_bin_4915.0',
   20: 'rx_bin_19595.0',
   21: 'rx_bin_1553.0',
   22: 'rx_bin_20107.0',
   23: 'rx_bin_610591.0',
   24: 'rx_bin_610455.0',
   25: 'rx_bin_610602.0',
   26: 'rx_bin_23880.0',
   27: 'rx_bin_10579.0',
   28: 'rx_bin_18902.0',
   29: 'rx_bin_610593.0',
   30: 'rx_bin_12312.0',
   31: 'rx_bin_3585.0',
   32: 'rx_pcn_unk',
   33: 'rx_pcn_minor',
   34: 'rx_pcn_ADV',
   35: 'rx_pcn_9999',
   36: 'rx_pcn_MEDDADV',
   37: 'rx_pcn_A4',
   38: 'rx_pcn_03200000',
   39: 'rx_pcn_MCAIDADV',
   40: '

In [47]:
ROName = list(ROName_to_ROds.keys())[0]
ROName

'Rx-ObsPnt-InsCate'

In [48]:
ROds   = ROName_to_ROds[ROName]
ROds

Dataset({
    features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
    num_rows: 1
})

In [49]:
if ROds is None:
    ROdf = pd.DataFrame()
else:
    ROdf = ROds.to_pandas()
ROdf

,PID,RxID,DT,Rx-InsCateTkn_wgt,Rx-InsCateTkn_tknidx
0,1027056,1027056-002-000,2023-07-17 20:54:55.528,"[1, 1, 1]","[1, 33, 65]"


In [50]:
# ROdf = ROds.to_pandas()

FldName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['FldName']
FldNameTkn = FldName + 'Tkn'

In [51]:
key_list = [i for i in ROdf.columns if FldNameTkn in i]
key_list

['Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx']

In [52]:
tknidx_col = [i for i in key_list if '_tknidx' in i][0]
tknidx_col

'Rx-InsCateTkn_tknidx'

In [53]:
wgt_col = [i for i in key_list if '_wgt' in i][0]
wgt_col

'Rx-InsCateTkn_wgt'

In [54]:
from recfldtkn.obsname import parse_RecObsName
import itertools
tknidx_seq = list(itertools.chain(*ROdf[tknidx_col].to_list()))
tknidx_seq

[1, 33, 65]

In [55]:
wgt_seq = list(itertools.chain(*ROdf[wgt_col].to_list()))
wgt_seq

[1, 1, 1]

In [56]:
df_feat = pd.DataFrame({'tknidx': tknidx_seq, 'wgt': wgt_seq})
df_feat

,tknidx,wgt
0,1,1
1,33,1
2,65,1


In [57]:
df_feat['tknidx'].value_counts()

tknidx
1     1
33    1
65    1
Name: count, dtype: int64

In [58]:
df_feat = df_feat.groupby('tknidx').sum().reset_index()
df_feat

,tknidx,wgt
0,1,1
1,33,1
2,65,1


In [59]:
tid = df_feat['tknidx'].to_list()
tid

[1, 33, 65]

In [60]:
wgt = df_feat['wgt'].to_list()
wgt

[1, 1, 1]

In [61]:
CO = {'tid': tid, 'wgt': wgt}
CO

{'tid': [1, 33, 65], 'wgt': [1, 1, 1]}

In [62]:
def fn_CasePhi(case_example, 
               ROName_to_ROds, 
               ROName_to_ROInfo, 
               CO_vocab, 
               cohort_args):
    
    from recfldtkn.obsname import parse_RecObsName
    import itertools
    
    # input: RecObsName_to_RecObsDS, RecObsName_to_RecObsInfo
    # output: CaseObservation
    assert len(ROName_to_ROds) == 1
    
    # RO
    ROName = list(ROName_to_ROds.keys())[0]
    # for RecObsName in RecObsName_to_RecObsDS:
    
    #############################################
    ROds   = ROName_to_ROds[ROName] # dataframe: RecObsName: Rx-bf24.. RecObsDS: the df: record collection
    ROInfo = ROName_to_ROInfo[ROName]
    
    d = {}
    if ROds is not None:
        ROdf = ROds.to_pandas()
        
        FldName = parse_RecObsName(ROName, cohort_args['ckpd_to_CkpdObsConfig'])['FldName']
        FldNameTkn = FldName + 'Tkn'
        
        key_list = [i for i in ROdf.columns if FldNameTkn in i]
        tknidx_col = [i for i in key_list if '_tknidx' in i][0]
        wgt_col = [i for i in key_list if '_wgt' in i][0]
        #############################################
        tknidx_seq = list(itertools.chain(*ROdf[tknidx_col].to_list()))
        wgt_seq = list(itertools.chain(*ROdf[wgt_col].to_list()))
        df_feat = pd.DataFrame({'tknidx': tknidx_seq, 'wgt': wgt_seq})
        df_feat = df_feat.groupby('tknidx').mean().reset_index()
        tid = df_feat['tknidx'].to_list()
        wgt = df_feat['wgt'].to_list()
        #############################################
        CO = {'tid': tid, 'wgt': wgt}

    else:
        tkn2tid = CO_vocab['tid']['tkn2tid']
        # d['NoObs'] = 1
        tkn = ['NoObs']
        tid = [tkn2tid[i] for i in tkn]
        wgt = [1]
        CO = {'tid': tid, 'wgt': wgt}
    #############################################
    return CO

fn_CasePhi.fn_string = inspect.getsource(fn_CasePhi)

In [63]:
ROName_to_ROds

{'Rx-ObsPnt-InsCate': Dataset({
     features: ['PID', 'RxID', 'DT', 'Rx-InsCateTkn_wgt', 'Rx-InsCateTkn_tknidx'],
     num_rows: 1
 })}

In [64]:
CO = fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab, cohort_args)

In [65]:
CO

{'tid': [1, 33, 65], 'wgt': [1.0, 1.0, 1.0]}

In [66]:
CO_vocab

{'tid': {'tid2tkn': {0: 'rx_bin_unk',
   1: 'rx_bin_minor',
   2: 'rx_bin_4336.0',
   3: 'rx_bin_610097.0',
   4: 'rx_bin_610014.0',
   5: 'rx_bin_3858.0',
   6: 'rx_bin_15581.0',
   7: 'rx_bin_610502.0',
   8: 'rx_bin_610011.0',
   9: 'rx_bin_11552.0',
   10: 'rx_bin_610494.0',
   11: 'rx_bin_610239.0',
   12: 'rx_bin_610279.0',
   13: 'rx_bin_20115.0',
   14: 'rx_bin_20099.0',
   15: 'rx_bin_17010.0',
   16: 'rx_bin_24251.0',
   17: 'rx_bin_22659.0',
   18: 'rx_bin_12833.0',
   19: 'rx_bin_4915.0',
   20: 'rx_bin_19595.0',
   21: 'rx_bin_1553.0',
   22: 'rx_bin_20107.0',
   23: 'rx_bin_610591.0',
   24: 'rx_bin_610455.0',
   25: 'rx_bin_610602.0',
   26: 'rx_bin_23880.0',
   27: 'rx_bin_10579.0',
   28: 'rx_bin_18902.0',
   29: 'rx_bin_610593.0',
   30: 'rx_bin_12312.0',
   31: 'rx_bin_3585.0',
   32: 'rx_pcn_unk',
   33: 'rx_pcn_minor',
   34: 'rx_pcn_ADV',
   35: 'rx_pcn_9999',
   36: 'rx_pcn_MEDDADV',
   37: 'rx_pcn_A4',
   38: 'rx_pcn_03200000',
   39: 'rx_pcn_MCAIDADV',
   40: '

## Save Files

In [67]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CO_id, get_selected_columns, get_CO_vocab, fn_CasePhi]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casephi', f'phi_{name_CasePhi}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [68]:
CaseObsName

'ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat'

In [69]:
from recfldtkn.observer import get_CaseObsInfo_for_a_CaseObsName

CaseObsInfo = get_CaseObsInfo_for_a_CaseObsName(CaseObsName,
                                                SPACE, 
                                                cohort_args, 
                                                record_to_ds_rec = {}, 
                                                record_to_ds_rec_info = {})
[i for i in CaseObsInfo]

[INFO:2024-04-09 12:19:18,763:(configfn.py@110 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline/config_recfldtkn/Record/Rx.yaml


['RecObsName_List',
 'name_CasePhi',
 'get_selected_columns',
 'CaseObsName',
 'ROName_to_ROInfo',
 'fn_CasePhi',
 'get_CO_id',
 'CO_Folder',
 'CO_vocab']

In [70]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


ro_to_ROName = {} # leave it empty for now 
ROName_to_ROInfo = CaseObsInfo['ROName_to_ROInfo']
name_CasePhi = CaseObsInfo['name_CasePhi']
fn_CasePhi = CaseObsInfo['fn_CasePhi']
CO_vocab = CaseObsInfo['CO_vocab']
get_CO_id = CaseObsInfo['get_CO_id']
CO_Folder = CaseObsInfo['CO_Folder']
COids = None 
use_CO_from_disk = False
batch_size = CaseObsInfo.get('batch_size', 1000)

fn_caseobs_Phi = CaseObserverTransformer(ro_to_ROName,
                                         ROName_to_ROInfo, 
                                         name_CasePhi, 
                                         fn_CasePhi, 
                                         CO_vocab, 
                                         get_CO_id,
                                         cohort_args,
                                         CO_Folder, 
                                         df_case,
                                         use_CO_from_disk)

# Check Cache Functions

In [71]:
fn_caseobs_Phi.new_COs

{}

In [72]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:04.148170
Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'DT', 'drug_name', 'tid', 'wgt'],
    num_rows: 1000
})


In [73]:
print(len(fn_caseobs_Phi.new_COs))

1000


In [74]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.027818
Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'DT', 'drug_name', 'tid', 'wgt'],
    num_rows: 1000
})


In [75]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CO_vocab)

ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat
404
{'DT': Timestamp('2023-06-14 03:23:53.558000'),
 'ObsDT': Timestamp('2023-06-14 03:23:53.558000'),
 'PID': 1010322,
 'PInvID': '1010322-007',
 'RxID': '1010322-007-011',
 'drug_name': 'Trulicity',
 'tid': [1, 33, 65],
 'wgt': [1.0, 1.0, 1.0]}
{'tid': {'tid2tkn': {0: 'rx_bin_unk', 1: 'rx_bin_minor', 2: 'rx_bin_4336.0', 3: 'rx_bin_610097.0', 4: 'rx_bin_610014.0', 5: 'rx_bin_3858.0', 6: 'rx_bin_15581.0', 7: 'rx_bin_610502.0', 8: 'rx_bin_610011.0', 9: 'rx_bin_11552.0', 10: 'rx_bin_610494.0', 11: 'rx_bin_610239.0', 12: 'rx_bin_610279.0', 13: 'rx_bin_20115.0', 14: 'rx_bin_20099.0', 15: 'rx_bin_17010.0', 16: 'rx_bin_24251.0', 17: 'rx_bin_22659.0', 18: 'rx_bin_12833.0', 19: 'rx_bin_4915.0', 20: 'rx_bin_19595.0', 21: 'rx_bin_1553.0', 22: 'rx_bin_20107.0', 23: 'rx_bin_610591.0', 24: 'rx_bin_610455.0', 25: 'rx_bin_610602.0', 26: 'rx_bin_23880.0', 27: 'rx_bin_10579.0', 28: 'rx_bin_18902.0', 29: 'rx_bin_610593.0', 30: 'rx_bin_12312.0', 31: 'rx_bin_3585.0', 32

In [76]:
ds_casetkn[1]

{'PID': 1009211,
 'ObsDT': Timestamp('2023-09-01 19:40:45.612000'),
 'PInvID': '1009211-001',
 'RxID': '1009211-001-000',
 'DT': Timestamp('2023-09-01 19:40:45.612000'),
 'drug_name': 'Trulicity',
 'tid': [2, 33, 65],
 'wgt': [1.0, 1.0, 1.0]}

## Save

In [77]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [78]:
CO_Folder_data = fn_caseobs_Phi.CO_Folder_data 
CO_Folder_data

'../_Data/2-Data_CaseObs/ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat/data'

In [79]:
fn_caseobs_Phi.save_new_COs_to_disk(CO_Folder_data)

[INFO:2024-04-09 12:19:24,187:(observer.py@691 recfldtkn.observer)]: use_CO_from_disk is false and empty new_COs


In [80]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [81]:
fn_caseobs_Phi.ds_CO_data

# Save Vocab

In [82]:
CO_vocab = fn_caseobs_Phi.CO_vocab
CO_vocab

{'tid': {'tid2tkn': {0: 'rx_bin_unk',
   1: 'rx_bin_minor',
   2: 'rx_bin_4336.0',
   3: 'rx_bin_610097.0',
   4: 'rx_bin_610014.0',
   5: 'rx_bin_3858.0',
   6: 'rx_bin_15581.0',
   7: 'rx_bin_610502.0',
   8: 'rx_bin_610011.0',
   9: 'rx_bin_11552.0',
   10: 'rx_bin_610494.0',
   11: 'rx_bin_610239.0',
   12: 'rx_bin_610279.0',
   13: 'rx_bin_20115.0',
   14: 'rx_bin_20099.0',
   15: 'rx_bin_17010.0',
   16: 'rx_bin_24251.0',
   17: 'rx_bin_22659.0',
   18: 'rx_bin_12833.0',
   19: 'rx_bin_4915.0',
   20: 'rx_bin_19595.0',
   21: 'rx_bin_1553.0',
   22: 'rx_bin_20107.0',
   23: 'rx_bin_610591.0',
   24: 'rx_bin_610455.0',
   25: 'rx_bin_610602.0',
   26: 'rx_bin_23880.0',
   27: 'rx_bin_10579.0',
   28: 'rx_bin_18902.0',
   29: 'rx_bin_610593.0',
   30: 'rx_bin_12312.0',
   31: 'rx_bin_3585.0',
   32: 'rx_pcn_unk',
   33: 'rx_pcn_minor',
   34: 'rx_pcn_ADV',
   35: 'rx_pcn_9999',
   36: 'rx_pcn_MEDDADV',
   37: 'rx_pcn_A4',
   38: 'rx_pcn_03200000',
   39: 'rx_pcn_MCAIDADV',
   40: '

In [83]:
CO_Folder_vocab = fn_caseobs_Phi.CO_Folder_vocab
CO_Folder_vocab

'../_Data/2-Data_CaseObs/ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat/vocab.p'

In [84]:
df_Vocab = pd.DataFrame({CaseObsName: CO_vocab})
df_Vocab.to_pickle(CO_Folder_vocab)
df_Vocab

,ro.Rx-ObsPnt-InsCate_ct.AggMeanFeat
tid,"{'tid2tkn': {0: 'rx_bin_unk', 1: 'rx_bin_minor..."
wgt,{}
